In [1]:
#Package
import ee
import folium
import geemap
import rasterio

In [2]:
ee.Authenticate()
ee.Initialize()

Enter verification code: 4/1gG21UECIAHMC3VAZVXtLO3NlW0yTBeCFgdeYeYmr6DrU7YrTzOFUIs

Successfully saved authorization token.


In [3]:
images = ee.ImageCollection("COPERNICUS/S2")\
            .filterBounds(ee.Geometry.Rectangle(110.7, -7, 110.9, -6.8))\
            .filterDate(ee.Date('2019-03-01'), ee.Date('2019-12-30'))\
            .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 5))\
            .sort('CLOUDY_PIXEL_PERCENTAGE').first()

In [4]:
NDVI = images.expression('(B8-B4)/(B8+B4)', {'B8': images.select('B8'),
                                            'B4': images.select('B4')}).rename('NDVI')


In [16]:
vis = {'bands': ['B4', 'B3', 'B2'],
      'min': 0,
      'max':3000}

vegvis = {'bands': ['B8', 'B4', 'B3'],
      'min': 0,
      'max':3000}

indeksvis = {'palette': ['red', 'yellow', 'green']}

In [17]:
Map = geemap.Map()

Map.centerObject(ee.Geometry.Rectangle(110.7, -7, 110.5, -6), zoom=12)
Map.addLayer(images, vis, 'Citra')
Map.addLayer(images, vegvis, 'Citra Veg')
Map.addLayer(NDVI, indeksvis, 'NDVI')
Map

Map(center=[-6.4998440994712965, 110.59999999999961], controls=(WidgetControl(options=['position'], widget=HBo…

In [91]:
NormalColor = geemap.ee_tile_layer(images, vis, 'Citra Semarang')
IndeksColor = geemap.ee_tile_layer(NDVI, indeksvis, 'NDVI')

In [93]:
smap = geemap.Map()
smap.centerObject(ee.Geometry.Rectangle(110.7, -7, 110.9, -6.8), zoom=10)

smap.split_map(NormalColor, IndeksColor)
smap

Map(center=[-6.900003367659871, 110.79999999999973], controls=(WidgetControl(options=['position'], widget=HBox…

In [42]:
bound = Map.draw_last_feature

In [49]:
bound.geometry().coordinates().getInfo()

[[[110.216708, -7.111386],
  [110.216708, -6.800035],
  [110.610623, -6.800035],
  [110.610623, -7.111386],
  [110.216708, -7.111386]]]

In [50]:
imgclip = images.clip(bound)

In [51]:
Map = geemap.Map()

Map.centerObject(bound, zoom=12)
Map.addLayer(imgclip, vis, 'Citra')
Map.addLayer(imgclip, vegvis, 'Citra Veg')
Map

Map(center=[-6.955708559267196, 110.41366549999931], controls=(WidgetControl(options=['position'], widget=HBox…

In [57]:
task = ee.batch.Export.image.toDrive(imgclip.select(['B8', 'B4', 'B3']), 'Semarang', folder='KotaSemarang', fileNamePrefix='CitraSemarangVeg', region=bound.geometry(), scale=10 )
task.start()